<a href="https://colab.research.google.com/github/punctuationmarks/Linux/blob/master/Setting_up_Linux_Server_(Ubuntu)_for_Web_(Flask)_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploying a linux server from scratch
Example being used is ssd drive through linode

### NOTE:  
#### Easiest way to do this is to have two bash terminals open at the same time, this way you aren't having to log back into your linux linode server , and back into your local machine , back and forth. 
#### For Windows machines, have one Ubuntu bash shell and one Windows terminal open.

## Logging In

After your create your Linode server,  take the ssh command and run it in linux bash 
 (could be ubuntu on windows, if you're doing that, but this HAS to be in bash)
 enter your password and you're in 
 should look something like this:


```
ssh root@12.34.56.78 

```



Once in, first thing update and upgrade your ubuntu with command:	


```
apt update && apt upgrade
```



## Setting up the hostname

after that happens, you're going to want to set your host name
 with command:
 
 

```
hostnamectl set-hostname PineappleApp-server
```
     



 double checkig to make sure it's updated
(this should return whatever you named your hostname)


```
hostname
```







 now you have to also set his new host name in your host file
 do this with the built in editor nano
 
```
nano /etc/hosts
```
 
          

 what you'll see is this:



```
127.0.0.1       localhost

The following lines are desirable for IPv6 capable hosts
::1     localhost ip6-localhost ip6-loopback
ff02::1 ip6-allnodes
ffo2::2 ip6-allrouters
```


This the only thing you add or change:
add your custom ip address under the "localhost" line and the new host name

```
12.34.56.78     PineappleApp-server
```



#### Note:
Nano keyboard use:



```
Control-X to exit, "y" to save and enter to commit changes
```



## Adding a user to the linode

next, you'll need to add a user with more limited access and commands 
(this is much safer than just running everything as admin)



```
adduser meetingthespam
```


 add a password for the user, and fill out anything else for it (everything else is optional)


 then you can add their privilages 
(sudo if you don't remember/didn't know if for root commands until restricted)


```
adduser meetingthespam sudo
```



to logout (you will see the bash shell back to local):



```
exit
```

 then you want to log out, best practice to not be logged into root
 you can log back into the ssh IP address by just replacing the root username



```
ssh meetingthespam@12.34.56.78
```


## Generating public/private rsa key pair (SSH)

next you'll want to set up SSH key authentication so you can log in without a password
and it's more secure because it's key-based authentication
make a .ssh directory inside the linux machine (linode in this case)
 
 
 ```
mkdir .ssh
```

 now on your local machine (not in the linode)
(either logout of your session with the linux server or just have two terminals open up)


 this generates the key and the "-b 4096" keeps it more secure
 
 
 
```
ssh-keygen -b 4096
```
          


 this prints:
 


  ```
   Generating public/private rsa key pair.
   Enter file in which to save the key (C:/Users/meetingthespam/.ssh/id_rsa):```




   you can leave the default if you'd like or make a new file name/location\
   NOTE: You will want to make a .ssh folder for each project you are using if you
   plan on having multiple projects on the same computer


   it'll prompt you for a passcode, do this if you'd want it more secure, 
   but it's not neccesary (and smoother if you don't)



   A public key will be generated and look something like 
   this if you kept the defualt settings 





```
Generating public/private rsa key pair.
Enter file in which to save the key (C:/Users/meetingthespam/.ssh/id_rsa):
Created directory 'C:/Users/meetingthespam/.ssh'.
Enter passphrase (empty for no passphrase):
Enter same passphrase again:
Your identification has been saved in /home/meetingthespam/.ssh/id_rsa.
Your public key has been saved in /home/meetingthespam/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:uAR18w/iogxrdppO7q44FiDIeNh8apmxdACA6JXqOg4 meetingthespam@DESKTOP-49Q9VPH
The key's randomart image is:
+---[RSA 4096]----+
|     .           |
|    + +          |
|     = o. .      |
|      . .o .     |
|     .  S.. * o  |
|.. . ...  .= E o |
|=.o.= .. ...o +o.|
|.*++.o.  ++ ..o..|
|oo=o.o  .+=o .  o|
+----[SHA256]-----+

```





## Moving the Public key to the server

now we need to move our public key to the server
go back into local bash/terminal and copy the public rsa key to the server 
 
Note on Syntax: * scp stands for secure copy program and the tidla (~) is for the home directory location, then the .ssh,  then a brand new file named "authorized_keys" (you don't need file extensions declared with linux)* 


In Mac/Linux:


```
scp ~/username/DirectoryYouSavedTheSSH/.ssh/id_rsa.pub meetingthespam@12.34.56.78:~/.ssh/authorized_keys
```




In Windows:



```
scp C:/Users/username/DirectoryYouSavedTheSSH/.ssh/id_rsa.pub meetingthespam@12.34.56.78:~/.ssh/authorized_keys
```




## Adding restrictions to use the SSH keys


 now back in your server bash, make sure the key is there 
 
 

```
ls .ssh
```





 
 this should show:
 
```
authorized_keys
```

 now for security, setting the permissions for the ssh directory 
 so that the owner has read, write, and execution permissions for any file
 and any owners of the files just have read, write:
 
 Synatx note: 
 *sudo chmod is changing the permissions command*




```
 sudo chmod 700 ~/.ssh/
 sudo chmod 600 ~/.ssh/*
```



Side note about these linux restrictions (700, 600, ect):
* the first digit of permission is for the user 
* the second digit is for the read, write and execute
* the third digit is for everyone else
        

 okay, now logout and log back in, it should just do it without needed a passcode 



## Changing setting to force SSH keys for login

 now that we know our ssh keys are working (allowing no passcode to login)
 we need to disallow root logins via ssh
 
 

```
sudo nano /etc/ssh/sshd_config
```







  this opens a pretty big file, for our use we just want to change two things:
 *  First:
 
```
PermitRootLogin no
```

*this is because since we have a user with limited access we technically don't need to login as root (throws off hackers since they might try to login via root)*


 
 * Second: (uncomment out and change "yes" to "no")
 
 

```
PasswordAuthentication no
```


*since we have our ssh keys working, sometimes hackers with try to brutforce a password , this prevents that from happening*
 
 
 
 
 After you save the sshd_config file, we'll want to restart our ssh server 
  
  ```
sudo systemctl restart sshd
```
  

  

 

## Uncomplicated Firewall

 now install an "uncomplicated firewall"
 
 ```
sudo apt install ufw
```

 
 
 now set up some rules for the firewall :
 allowing outgoing traffic
 
 
```
sudo ufw default allow outgoing
```


 denying incoming traffic 
 

```
sudo ufw default deny incoming
```


 TO MAKE SURE YOU DON'T GET LOCKED OUT OF YOUR SERVER
 allow incoming into certain ports 
 like ssh, and http and any port we want to access outside of our server 

 allows us to use ssh to login 
 
 
 ```
 sudo ufw allow ssh
 ```
 

 since flask uses a port 5000 to debug, we'll need to allow it for now (Django uses port 8000)

```
sudo ufw allow 5000
```


 next we need to impliment all of those changes, be very careful here 
 (meaning make sure you at least allow ssh login, if that's what you're using)
 

```
sudo ufw enable
```

This prints, press "y" for the firewall to activate:



```
Command may disrupt existing ssh connections. Proceed with operation (y|n)? y
Firewall is active and enabled on system startup
```




### Getting stressed that you might have forgotten something?
 check the status:
 

```
sudo ufw status
```



This should print:



```
Status: active

To                         Action      From
--                         ------      ----
22/tcp                     ALLOW       Anywhere
5000                       ALLOW       Anywhere
22/tcp (v6)                ALLOW       Anywhere (v6)
5000 (v6)                  ALLOW       Anywhere (v6)

port 22 is ssh
```



 #### Side Note:
 Want to spin up a linux server quicker through Linode.com?
 click on Create New Linode, Create from StackScript
 they have a bunch, like wordpress for example

